In [13]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import ydf
import pandas as pd
import numpy as np
import tensorflow as tf
import tf_keras
import math

In [17]:
dataset_df = pd.read_csv("data/noid_cvd.csv", sep=',')


In [32]:
# 将数据集分割为训练集和测试集。

def split_dataset(dataset, test_ratio=0.005):
  """将panda数据框分割成两部分。"""
  # 生成一个与数据集长度相同的随机数组，元素值小于测试比例的为True，大于等于测试比例的为False
  test_indices = np.random.rand(len(dataset)) < test_ratio
  # 返回测试集和训练集
  return dataset[~test_indices], dataset[test_indices]


# 调用split_dataset函数将数据集分割成训练集和测试集，并将返回的结果分别赋值给train_ds_pd和test_ds_pd
train_ds_pd, test_ds_pd = split_dataset(dataset_df)

# 打印训练集和测试集的样本数量
print("{}个样本用于训练，{}个样本用于测试。".format(
    len(train_ds_pd), len(test_ds_pd)))


69635个样本用于训练，365个样本用于测试。


In [38]:
tuner = ydf.RandomSearchTuner(num_trials=50, automatic_search_space=True)
model = ydf.RandomForestLearner(label="cardio").train(train_ds_pd)
evaluation = model.evaluate(test_ds_pd)
model_2 = ydf.GradientBoostedTreesLearner(label="cardio",tuner=tuner).train(train_ds_pd)
evaluation_2 = model_2.evaluate(test_ds_pd)
# model_3 = ydf.RandomForestLearner(
#     label="cardio",
#     # Tell the Random Forest to compute Out-of-bag variable importances.
#     # This is expensive but gives the best results.
#     compute_oob_variable_importances=True,
#     # Enable feature selection using the "backward selection" algorithm.
#     feature_selector=ydf.BackwardSelectionFeatureSelector(),
# ).train(train_ds_pd)

# evaluation_3 = model_3.evaluate(test_ds_pd)
print(evaluation)
print(evaluation_2)
# print(evaluation_3)


Train model on 69635 examples
Model trained in 0:00:05.918148
Train model on 69635 examples
Model trained in 0:39:13.220576
accuracy: 0.731507
confusion matrix:
    label (row) \ prediction (col)
    +-----+-----+-----+
    |     |   0 |   1 |
    +-----+-----+-----+
    |   0 | 146 |  42 |
    +-----+-----+-----+
    |   1 |  56 | 121 |
    +-----+-----+-----+
characteristics:
    name: '1' vs others
    ROC AUC: 0.805731
    PR AUC: 0.788246
    Num thresholds: 162
loss: 1.00976
num examples: 365
num examples (weighted): 365

accuracy: 0.736986
confusion matrix:
    label (row) \ prediction (col)
    +-----+-----+-----+
    |     |   0 |   1 |
    +-----+-----+-----+
    |   0 | 145 |  43 |
    +-----+-----+-----+
    |   1 |  53 | 124 |
    +-----+-----+-----+
characteristics:
    name: '1' vs others
    ROC AUC: 0.807399
    PR AUC: 0.793523
    Num thresholds: 365
loss: 0.534097
num examples: 365
num examples (weighted): 365



In [41]:

model_2.describe()

trial,score,duration,split_axis,sparse_oblique_projection_density_factor,sparse_oblique_normalization,sparse_oblique_weights,categorical_algorithm,growing_strategy,max_num_nodes,sampling_method,subsample,shrinkage,min_examples,use_hessian_gain,num_candidate_attributes_ratio,max_depth
37,-1.06542,1866.7,AXIS_ALIGNED,,,,RANDOM,LOCAL,,RANDOM,0.8,0.02,5,false,0.9,6
42,-1.06671,2080.01,SPARSE_OBLIQUE,2,MIN_MAX,BINARY,CART,BEST_FIRST_GLOBAL,32,RANDOM,0.9,0.1,10,false,0.2,
5,-1.06688,206.245,SPARSE_OBLIQUE,1,MIN_MAX,BINARY,RANDOM,BEST_FIRST_GLOBAL,32,RANDOM,0.6,0.02,20,false,1,
14,-1.06704,631.199,SPARSE_OBLIQUE,5,NONE,BINARY,CART,BEST_FIRST_GLOBAL,256,RANDOM,0.8,0.02,20,true,1,
21,-1.06709,981.216,SPARSE_OBLIQUE,1,STANDARD_DEVIATION,BINARY,RANDOM,LOCAL,,RANDOM,0.9,0.02,7,true,1,6
3,-1.06711,140.915,SPARSE_OBLIQUE,1,NONE,BINARY,CART,BEST_FIRST_GLOBAL,64,RANDOM,1,0.1,20,false,0.2,
34,-1.06737,1800.69,SPARSE_OBLIQUE,4,NONE,BINARY,RANDOM,LOCAL,,RANDOM,0.9,0.02,7,true,0.5,6
8,-1.0675,322.894,SPARSE_OBLIQUE,3,STANDARD_DEVIATION,CONTINUOUS,RANDOM,BEST_FIRST_GLOBAL,256,RANDOM,0.9,0.05,10,true,0.9,
0,-1.06753,31.7924,SPARSE_OBLIQUE,5,MIN_MAX,BINARY,CART,BEST_FIRST_GLOBAL,32,RANDOM,1,0.1,10,false,0.2,
1,-1.06753,95.578,SPARSE_OBLIQUE,2,NONE,BINARY,RANDOM,BEST_FIRST_GLOBAL,128,RANDOM,0.8,0.02,10,true,0.9,
